In [ ]:
import tensorflow as tf
import numpy as np
import cifar10_input
import cifar10_model
import matplotlib.pyplot as plt
import os
from datetime import datetime
# 验证
def evaluation():
    with tf.Graph().as_default():
#         print(BATCH_SIZE)
        n_test = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL
        eval_images, eval_lables = cifar10_input.inputs('/root/code/cifar_10/cifar10-CNN-master/cifar10_data/cifar-10-batches-bin',128)
        print('这是eval_images,eval_lables ',eval_images,eval_lables)
        eval_logits = cifar10_model.inference(eval_images)
        print('构建完网络',eval_logits)
        result = tf.nn.softmax(eval_logits)
        print("这是预测结果矩阵",result)
        # tf.nn.in_top_k(predictions, targets, k, name=None)
        # 每个样本的预测结果的前k个最大的数里面是否包括包含targets预测中的标签，一般取1，
        # 即取预测最大概率的索引与标签的对比
        top_k_op = tf.nn.in_top_k(eval_logits, eval_lables, 1)
        print('这是top_k_op  ',top_k_op)
        saver = tf.train.Saver()
        with tf.Session() as session:
            ckpt = tf.train.get_checkpoint_state('signal_GPU/saver')
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(session, ckpt.model_checkpoint_path)
            
#             predints1=session.run(eval_logits)
#             print('这是预测1',predints1)
            predints2=session.run(result)
            print('这是预测2',predints2)


            
#             result.append(str(predints[0]))
            
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=session, coord=coord)
            num_iter = int(n_test / BATCH_SIZE)
            true_count = 0
            for step in range(num_iter):
                predictions = session.run(top_k_op)
                print('这是predictions',predictions)
                true_count = true_count + np.sum(predictions)
            precision = true_count / (num_iter * BATCH_SIZE)
            print('precision=', precision)
            coord.request_stop()
            coord.join(threads)
evaluation()    


这是eval_images,eval_lables  Tensor("batch:0", shape=(128, 24, 24, 3), dtype=float32) Tensor("Reshape:0", shape=(128,), dtype=int32)
构建完网络 Tensor("softmax_linear/logits:0", shape=(128, 10), dtype=float32)
这是预测结果矩阵 Tensor("Softmax:0", shape=(128, 10), dtype=float32)
这是top_k_op   Tensor("in_top_k/InTopKV2:0", shape=(128,), dtype=bool)
INFO:tensorflow:Restoring parameters from signal_GPU/saver/model.ckpt


In [ ]:
from PIL import Image, ImageFilter
import tensorflow as tf
import numpy as np
import cifar10_input
import cifar10_model
import matplotlib.pyplot as plt
import os
from datetime import datetime
eval_images, eval_lables = cifar10_input.inputs('/root/code/cifar_10/cifar10-CNN-master/cifar10_data/cifar-10-batches-bin',128)

with tf.name_scope('conv1'):
    conv1_w = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], mean=0, stddev=5e-2), name='weights')
    conv1_b = tf.Variable(tf.zeros(64), name='biases')
    conv1 = tf.nn.relu(tf.nn.conv2d(eval_images, conv1_w, strides=[1, 1, 1, 1], padding='SAME') + conv1_b)
tf.summary.histogram('conv1_w', conv1_w)
tf.summary.histogram('conv1_b', conv1_b)

# pooling_1
pooling_1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pooling_1')
# norm1
norm1 = tf.nn.lrn(pooling_1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm_1')

# conv2
with tf.name_scope('conv2'):
    conv2_w = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], mean=0, stddev=5e-2), name='weights')
    conv2_b = tf.Variable(tf.zeros(64), name='biases')
    conv2 = tf.nn.relu(tf.nn.conv2d(norm1, conv2_w, strides=[1, 1, 1, 1], padding='SAME') + conv2_b)
tf.summary.histogram('conv2_w', conv2_w)
tf.summary.histogram('conv2_b', conv2_b)

# pooling_2
pooling_2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pooling_2')
# norm 2
norm2 = tf.nn.lrn(pooling_2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm_2')

#
reshaped_output = tf.reshape(norm2, shape=[128, -1])
dim = reshaped_output.shape[1].value

# local 3
with tf.name_scope('local3'):
    local3_w = tf.Variable(tf.truncated_normal(shape=[dim, 384], mean=0, stddev=0.04), name='weights')
    local3_b = tf.Variable(tf.zeros(384), name='biases')
    local3 = tf.nn.relu(tf.matmul(reshaped_output, local3_w) + local3_b)
tf.summary.histogram('local3_w', local3_w)
tf.summary.histogram('local3_b', local3_b)

# local 4
with tf.name_scope('local4'):
    local4_w = tf.Variable(tf.truncated_normal(shape=[384, 192], mean=0, stddev=0.04), name='weights')
    local4_b = tf.Variable(tf.zeros(192), name='biases')
    local4 = tf.nn.relu(tf.matmul(local3, local4_w) + local4_b)
tf.summary.histogram('local4_w', local4_w)
tf.summary.histogram('local4_b', local4_b)

# softmax_linear
with tf.name_scope('softmax_linear'):
    output_w = tf.Variable(tf.truncated_normal(shape=[192, 10], mean=0, stddev=1 / 192.0))
    output_b = tf.Variable(tf.zeros(10))
#         logits = tf.nn.relu(tf.matmul(local4, output_w) + output_b,name='logits')
    logits = tf.nn.softmax(tf.matmul(local4, output_w) + output_b,name='logits')






with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('signal_GPU/saver/model.ckpt.meta')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,'signal_GPU/saver/model.ckpt')

#     sess.run(tf.global_variables_initializer())
#     saver.restore(sess, 'signal_GPU/saver/model.ckpt') #使用模型，参数和之前的代码保持一致

    
    n_test = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL
    eval_images, eval_lables = cifar10_input.inputs('/root/code/cifar_10/cifar10-CNN-master/cifar10_data/cifar-10-batches-bin',128)
    print('这是eval_images,eval_lables ',eval_images,eval_lables)
    eval_logits = cifar10_model.inference(eval_images)
    print('构建完网络',eval_logits)
    predints=sess.run(logits)
    print(predints)


def predict10(sess, adds):
    import json
    num_imgs = len(adds)
    result = []
    for i in range(num_imgs):
        data = json.loads(adds[i])
        predints=sess.run('y_conv:0',feed_dict={"x:0":data,"keep_prob:0": 1.0})
        result.append(str(predints[0]))
    
    return result

INFO:tensorflow:Restoring parameters from signal_GPU/saver/model.ckpt
这是eval_images,eval_lables  Tensor("batch_6:0", shape=(128, 24, 24, 3), dtype=float32) Tensor("Reshape_14:0", shape=(128,), dtype=int32)
构建完网络 Tensor("softmax_linear_5/logits:0", shape=(128, 10), dtype=float32)
